# Forest Fire Prediction Model Training

This notebook demonstrates the training and evaluation of fire prediction models (U-Net and LSTM) using the forest fire prediction pipeline.

## Contents
1. Setup and Configuration
2. Data Preprocessing
3. Model Training (U-Net)
4. Model Training (LSTM)
5. Model Evaluation and Comparison
6. Prediction Visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os
import sys
import yaml
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path
sys.path.append('../src')

# Import custom modules
from utils import load_config, create_output_directories
from data_preprocessing import FireDataPreprocessor
from fire_prediction_model import UNetFirePredictor, LSTMFirePredictor, ModelEvaluator

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
print("Setup completed successfully!")

## 1. Setup and Configuration

In [ ]:
# Load configuration
config_path = '../config/default_config.yaml'

try:
    config = load_config(config_path)
    print("Configuration loaded successfully")
    
    # Display key configuration parameters
    print("\nKey Configuration Parameters:")
    print(f"  Model Type: {config.get('model_type', 'unet')}")
    print(f"  Target Resolution: {config.get('target_resolution', 30)} meters")
    print(f"  Batch Size: {config.get('batch_size', 16)}")
    print(f"  Epochs: {config.get('epochs', 100)}")
    print(f"  Learning Rate: {config.get('learning_rate', 0.001)}")
    
except Exception as e:
    print(f"Error loading configuration: {e}")
    # Create default configuration for demonstration
    config = {
        'model_type': 'unet',
        'target_resolution': 30.0,
        'batch_size': 8,
        'epochs': 50,
        'learning_rate': 0.001,
        'input_shape': [256, 256, 10],
        'paths': {
            'outputs': '../data/outputs',
            'models': '../models',
            'processed_data': '../data/processed'
        }
    }
    print("Using default configuration for demonstration")

# Create output directories
output_dirs = create_output_directories(config['paths']['outputs'])
config['output_dirs'] = output_dirs

print(f"\nOutput directories created at: {config['paths']['outputs']}")

## 2. Generate Sample Training Data

In [ ]:
def generate_sample_training_data(n_samples=1000, n_features=10, image_size=(64, 64)):
    """Generate sample data for model training demonstration."""
    
    print(f"Generating {n_samples} samples with {n_features} features...")
    
    # For tabular models (LSTM, simple models)
    X_tabular = np.random.randn(n_samples, n_features)
    
    # Create fire probability based on features
    # Higher temperature, lower humidity, higher wind speed increase fire probability
    fire_prob = (
        0.3 * X_tabular[:, 0] +      # temperature effect
        -0.2 * X_tabular[:, 1] +     # humidity effect (negative)
        0.15 * X_tabular[:, 2] +     # wind speed effect
        0.1 * X_tabular[:, 3] +      # fuel load effect
        np.random.randn(n_samples) * 0.1  # noise
    )
    
    # Convert to binary labels
    y_tabular = (fire_prob > np.percentile(fire_prob, 90)).astype(int)  # Top 10% as fire
    
    # For U-Net (image patches)
    n_patches = n_samples // 4  # Fewer patches due to memory constraints
    X_image = np.random.randn(n_patches, image_size[0], image_size[1], n_features)
    
    # Create spatial fire patterns
    y_image = np.zeros((n_patches, image_size[0], image_size[1], 1))
    
    for i in range(n_patches):
        # Create random fire spots
        n_fires = np.random.randint(0, 3)
        for _ in range(n_fires):
            center_x = np.random.randint(10, image_size[0] - 10)
            center_y = np.random.randint(10, image_size[1] - 10)
            radius = np.random.randint(3, 8)
            
            # Create circular fire pattern
            y, x = np.ogrid[:image_size[0], :image_size[1]]
            mask = (x - center_x)**2 + (y - center_y)**2 <= radius**2
            y_image[i, mask, 0] = 1
    
    print(f"Generated data shapes:")
    print(f"  Tabular: X {X_tabular.shape}, y {y_tabular.shape}")
    print(f"  Image: X {X_image.shape}, y {y_image.shape}")
    print(f"  Fire percentage (tabular): {np.mean(y_tabular)*100:.1f}%")
    print(f"  Fire percentage (image): {np.mean(y_image)*100:.1f}%")
    
    return X_tabular, y_tabular, X_image, y_image

# Generate sample data
X_tab, y_tab, X_img, y_img = generate_sample_training_data()

# Split data for training and validation
from sklearn.model_selection import train_test_split

# Split tabular data
X_tab_train, X_tab_test, y_tab_train, y_tab_test = train_test_split(
    X_tab, y_tab, test_size=0.2, random_state=42, stratify=y_tab)

X_tab_train, X_tab_val, y_tab_train, y_tab_val = train_test_split(
    X_tab_train, y_tab_train, test_size=0.2, random_state=42, stratify=y_tab_train)

# Split image data
X_img_train, X_img_test, y_img_train, y_img_test = train_test_split(
    X_img, y_img, test_size=0.2, random_state=42)

X_img_train, X_img_val, y_img_train, y_img_val = train_test_split(
    X_img_train, y_img_train, test_size=0.2, random_state=42)

print("\nData split completed:")
print(f"  Tabular - Train: {X_tab_train.shape}, Val: {X_tab_val.shape}, Test: {X_tab_test.shape}")
print(f"  Image - Train: {X_img_train.shape}, Val: {X_img_val.shape}, Test: {X_img_test.shape}")

## 3. U-Net Model Training

In [ ]:
# Configure U-Net model
unet_config = config.copy()
unet_config.update({
    'model_type': 'unet',
    'input_shape': [64, 64, 10],  # Match our sample data
    'n_classes': 2,
    'batch_size': 8,
    'epochs': 20,  # Reduced for demo
    'learning_rate': 0.001
})

print("Training U-Net Model...")
print("=" * 30)

# Initialize U-Net model
unet_model = UNetFirePredictor(unet_config)

# Build model
model = unet_model.build_model()
print(f"\nU-Net model built with input shape: {unet_config['input_shape']}")
print(f"Total parameters: {model.count_params():,}")

# Display model summary
model.summary()

In [ ]:
# Train U-Net model
model_save_path = os.path.join(config['paths']['models'], 'unet_demo_model.h5')
os.makedirs(config['paths']['models'], exist_ok=True)

try:
    # Train model
    history_unet = unet_model.train(
        X_img_train, y_img_train,
        X_img_val, y_img_val,
        model_save_path
    )
    
    print("\nU-Net training completed successfully!")
    
except Exception as e:
    print(f"Error during U-Net training: {e}")
    history_unet = None

## 4. LSTM Model Training

In [ ]:
# Configure LSTM model
lstm_config = config.copy()
lstm_config.update({
    'model_type': 'lstm',
    'sequence_length': 7,
    'n_features': 10,
    'hidden_size': 64,  # Reduced for demo
    'n_layers': 2,
    'batch_size': 16,
    'epochs': 20,  # Reduced for demo
    'learning_rate': 0.001
})

print("Training LSTM Model...")
print("=" * 30)

# Initialize LSTM model
lstm_model = LSTMFirePredictor(lstm_config)

# Prepare sequential data for LSTM
print("Preparing sequential data for LSTM...")

def create_sequences(X, y, sequence_length=7):
    """Create sequences for LSTM training."""
    X_seq, y_seq = [], []
    
    for i in range(sequence_length, len(X)):
        X_seq.append(X[i-sequence_length:i])
        y_seq.append(y[i])
    
    return np.array(X_seq), np.array(y_seq)

# Create sequences
X_lstm_train, y_lstm_train = create_sequences(X_tab_train, y_tab_train, lstm_config['sequence_length'])
X_lstm_val, y_lstm_val = create_sequences(X_tab_val, y_tab_val, lstm_config['sequence_length'])
X_lstm_test, y_lstm_test = create_sequences(X_tab_test, y_tab_test, lstm_config['sequence_length'])

print(f"LSTM sequences created:")
print(f"  Train: {X_lstm_train.shape}")
print(f"  Val: {X_lstm_val.shape}")
print(f"  Test: {X_lstm_test.shape}")

# Build LSTM model
lstm_model_tf = lstm_model.build_model()
print(f"\nLSTM model built")
print(f"Total parameters: {lstm_model_tf.count_params():,}")

# Display model summary
lstm_model_tf.summary()

In [ ]:
# Train LSTM model
lstm_model_save_path = os.path.join(config['paths']['models'], 'lstm_demo_model.h5')

try:
    # Train model
    history_lstm = lstm_model.train(
        X_lstm_train, y_lstm_train,
        X_lstm_val, y_lstm_val,
        lstm_model_save_path
    )
    
    print("\nLSTM training completed successfully!")
    
except Exception as e:
    print(f"Error during LSTM training: {e}")
    history_lstm = None

## 5. Model Evaluation and Comparison

In [ ]:
# Initialize evaluator
evaluator = ModelEvaluator()

# Evaluate U-Net model
if history_unet is not None:
    print("Evaluating U-Net Model...")
    print("=" * 30)
    
    # Make predictions
    y_pred_unet_proba = unet_model.predict(X_img_test)
    y_pred_unet = (y_pred_unet_proba > 0.5).astype(int)
    
    # Flatten for evaluation
    y_true_unet_flat = y_img_test.flatten()
    y_pred_unet_flat = y_pred_unet.flatten()
    y_pred_unet_proba_flat = y_pred_unet_proba.flatten()
    
    # Evaluate
    unet_metrics = evaluator.evaluate_model(
        y_true_unet_flat, y_pred_unet_flat, y_pred_unet_proba_flat,
        save_path=os.path.join(config['output_dirs']['metrics'], 'unet_metrics.json')
    )
    
    # Plot training history
    evaluator.plot_training_history(
        history_unet, 
        save_path=os.path.join(config['output_dirs']['visualizations'], 'unet_training_history.png')
    )
    
    # Plot confusion matrix
    evaluator.plot_confusion_matrix(
        y_true_unet_flat, y_pred_unet_flat,
        save_path=os.path.join(config['output_dirs']['visualizations'], 'unet_confusion_matrix.png')
    )

# Evaluate LSTM model
if history_lstm is not None:
    print("\nEvaluating LSTM Model...")
    print("=" * 30)
    
    # Make predictions
    y_pred_lstm_proba = lstm_model.model.predict(X_lstm_test)
    y_pred_lstm = (y_pred_lstm_proba > 0.5).astype(int)
    
    # Evaluate
    lstm_metrics = evaluator.evaluate_model(
        y_lstm_test, y_pred_lstm.flatten(), y_pred_lstm_proba.flatten(),
        save_path=os.path.join(config['output_dirs']['metrics'], 'lstm_metrics.json')
    )
    
    # Plot training history
    evaluator.plot_training_history(
        history_lstm,
        save_path=os.path.join(config['output_dirs']['visualizations'], 'lstm_training_history.png')
    )
    
    # Plot confusion matrix
    evaluator.plot_confusion_matrix(
        y_lstm_test, y_pred_lstm.flatten(),
        save_path=os.path.join(config['output_dirs']['visualizations'], 'lstm_confusion_matrix.png')
    )

In [ ]:
# Model comparison
if history_unet is not None and history_lstm is not None:
    print("\nModel Comparison:")
    print("=" * 40)
    
    comparison_data = {
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC AUC'],
        'U-Net': [
            unet_metrics['accuracy'],
            unet_metrics['precision'],
            unet_metrics['recall'],
            unet_metrics['f1_score'],
            unet_metrics.get('roc_auc', 0)
        ],
        'LSTM': [
            lstm_metrics['accuracy'],
            lstm_metrics['precision'],
            lstm_metrics['recall'],
            lstm_metrics['f1_score'],
            lstm_metrics.get('roc_auc', 0)
        ]
    }
    
    import pandas as pd
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.round(4))
    
    # Plot comparison
    fig, ax = plt.subplots(figsize=(10, 6))
    
    x = np.arange(len(comparison_data['Metric']))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, comparison_data['U-Net'], width, label='U-Net', alpha=0.8)
    bars2 = ax.bar(x + width/2, comparison_data['LSTM'], width, label='LSTM', alpha=0.8)
    
    ax.set_xlabel('Metrics')
    ax.set_ylabel('Score')
    ax.set_title('Model Performance Comparison')
    ax.set_xticks(x)
    ax.set_xticklabels(comparison_data['Metric'])
    ax.legend()
    ax.set_ylim(0, 1)
    
    # Add value labels on bars
    def add_value_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                   f'{height:.3f}', ha='center', va='bottom', fontsize=9)
    
    add_value_labels(bars1)
    add_value_labels(bars2)
    
    plt.tight_layout()
    plt.savefig(os.path.join(config['output_dirs']['visualizations'], 'model_comparison.png'), 
                dpi=300, bbox_inches='tight')
    plt.show()

## 6. Prediction Visualization

In [ ]:
# Visualize U-Net predictions
if history_unet is not None:
    print("Visualizing U-Net Predictions...")
    
    # Select some test samples for visualization
    n_samples = min(4, len(X_img_test))
    sample_indices = np.random.choice(len(X_img_test), n_samples, replace=False)
    
    fig, axes = plt.subplots(n_samples, 4, figsize=(16, 4*n_samples))
    if n_samples == 1:
        axes = axes.reshape(1, -1)
    
    for i, idx in enumerate(sample_indices):
        # Input features (show first channel)
        axes[i, 0].imshow(X_img_test[idx, :, :, 0], cmap='viridis')
        axes[i, 0].set_title(f'Input Features (Sample {idx})')
        axes[i, 0].axis('off')
        
        # True label
        axes[i, 1].imshow(y_img_test[idx, :, :, 0], cmap='Reds', vmin=0, vmax=1)
        axes[i, 1].set_title('True Fire Map')
        axes[i, 1].axis('off')
        
        # Predicted probability
        pred_prob = y_pred_unet_proba[idx, :, :, 0]
        axes[i, 2].imshow(pred_prob, cmap='Reds', vmin=0, vmax=1)
        axes[i, 2].set_title('Predicted Probability')
        axes[i, 2].axis('off')
        
        # Predicted binary
        pred_binary = y_pred_unet[idx, :, :, 0]
        axes[i, 3].imshow(pred_binary, cmap='Reds', vmin=0, vmax=1)
        axes[i, 3].set_title('Predicted Fire Map')
        axes[i, 3].axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(config['output_dirs']['visualizations'], 'unet_predictions.png'), 
                dpi=300, bbox_inches='tight')
    plt.show()

# Visualize LSTM predictions
if history_lstm is not None:
    print("\nVisualizing LSTM Predictions...")
    
    # Plot prediction vs actual for time series
    n_plot = min(100, len(y_lstm_test))
    
    plt.figure(figsize=(15, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(y_lstm_test[:n_plot], 'o-', label='Actual', alpha=0.7, markersize=4)
    plt.plot(y_pred_lstm_proba[:n_plot], 's-', label='Predicted Probability', alpha=0.7, markersize=3)
    plt.xlabel('Time Step')
    plt.ylabel('Fire Occurrence')
    plt.title('LSTM Predictions vs Actual')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.scatter(y_lstm_test, y_pred_lstm_proba, alpha=0.6)
    plt.plot([0, 1], [0, 1], 'r--', label='Perfect Prediction')
    plt.xlabel('Actual Fire Occurrence')
    plt.ylabel('Predicted Probability')
    plt.title('Predicted vs Actual Scatter Plot')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(config['output_dirs']['visualizations'], 'lstm_predictions.png'), 
                dpi=300, bbox_inches='tight')
    plt.show()

## Summary and Next Steps

This notebook demonstrated the training and evaluation of both U-Net and LSTM models for forest fire prediction:

### Key Accomplishments:
1. **Model Setup**: Configured and built both U-Net and LSTM architectures
2. **Training**: Trained models on sample data with proper validation
3. **Evaluation**: Comprehensive performance assessment with multiple metrics
4. **Visualization**: Created plots for training history, confusion matrices, and predictions
5. **Comparison**: Side-by-side comparison of model performance

### Model Characteristics:
- **U-Net**: Better for spatial fire patterns and pixel-level predictions
- **LSTM**: Better for temporal patterns and time-series prediction

### Next Steps:
1. **Real Data**: Replace sample data with actual environmental and fire history data
2. **Hyperparameter Tuning**: Optimize model parameters for better performance
3. **Ensemble Methods**: Combine both models for improved predictions
4. **Feature Engineering**: Add more relevant environmental features
5. **Cross-Validation**: Implement more robust validation strategies

### Files Generated:
- Model weights: `models/unet_demo_model.h5`, `models/lstm_demo_model.h5`
- Evaluation metrics: `data/outputs/metrics/`
- Visualizations: `data/outputs/visualizations/`

**Note**: This demonstration uses synthetic data. For production use, ensure you have:
- High-quality environmental data (weather, terrain, vegetation)
- Accurate historical fire occurrence data
- Proper data preprocessing and alignment
- Sufficient computational resources for larger datasets